In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity



import os
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.preprocessing as pp

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/256 Project/train_triplets.txt', sep='\t', index_col=None, names = ['User','Song', 'Count'])
# df = pd.read_csv('/content/drive/MyDrive/Fall 2022/CMPE 256/Project/train_triplets.txt', sep='\t', index_col=None, names = ['User','Song', 'Count'])
df.head()

,User,Song,Count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [ ]:
# https://gist.github.com/TomLin/918f246e14bc4a34c45461d862a42141
def df2interact_mat(df, user_col, item_col, interact_col):
	users = np.array(list(np.sort(
		df[user_col].unique())))  # Get our unique users

	items = np.array(list(
		df[item_col].unique()))  # Get our unique items that were interacted with

	interactions = list(
		df[interact_col])  # All of our interactions

	# Re-code user and item index for sparse matrix.
	# Index starting from 0
	rows = df[user_col].astype(CategoricalDtype(
		categories=users)).cat.codes

	cols = df[item_col].astype(CategoricalDtype(
		categories=items)).cat.codes

	# Construct sparse matrix.
	# Get the associated column indices
	interaction_sparse = sparse.csc_matrix((interactions, (rows, cols)), shape=(len(users), len(items)))

	return interaction_sparse, users, items

In [ ]:
mat, users, items = df2interact_mat(df, 'User','Song','Count')

In [ ]:
# calculate similarity between each row (user x songs)
def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat.tocsc(), axis=0)
    return col_normed_mat.T * col_normed_mat

In [ ]:
similarities = cosine_similarities(mat)

In [ ]:
def top_n_idx(matrix, n):
  # Return index of top n values in each row of a sparse matrix
  top_n_idx = []
  for l, r in zip(matrix.indptr[:-1], matrix.indptr[1:]):
    n_row = min(n, r - l)
    top_n_idx.append(matrix.indices[l + np.argpartition(matrix.data[l:r], -n_row)[-n_row:]])
  return top_n_idx

user_user_similar = top_n_idx(similarities, 5)

In [ ]:
# Saving top 5 user similarity file
with open('/content/drive/MyDrive/256 Project/user_user_similar.pkl', 'wb') as f:
  pickle.dump(user_user_similar, f)

In [ ]:
# Saving similarity matrix
with open('/content/drive/MyDrive/256 Project/similarity_matrix.pkl', 'wb') as f:
  pickle.dump(similarities, f, protocol=4)

In [ ]:
# Guide to loading file
with open('/content/drive/MyDrive/256 Project/user_user_similar.pkl', 'rb') as f:
  temp = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/256 Project/similarity_matrix.pkl', 'rb') as f:
  pickle.dump(similarities, f)